In [1]:
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from tqdm._tqdm_notebook import tqdm_notebook

Using TensorFlow backend.


In [2]:
data = pd.read_csv(r"E:\EEG Analysis\metas\train.csv")
data.head()

,Unnamed: 0,locations,label
0,0,E:/EEG Analysis/train/0.csv,0
1,1,E:/EEG Analysis/train/1.csv,0
2,2,E:/EEG Analysis/train/2.csv,0
3,3,E:/EEG Analysis/train/3.csv,0
4,4,E:/EEG Analysis/train/4.csv,0


## Read each file, break it into a 1 second window(1000 readings)
### Store the new file into a new csv with the same label of it's parent. 
### Create a new dataframe with the info related to the same

In [13]:
def check_times(location):
    data = pd.read_csv(location, header=None)
    print(data.shape)
    
#data['locations'].apply(lambda x: check_times(x))

### We observe that the length of each data file is not divisible by 1000. So we'll need to pad the sequences

In [50]:
#Function to read the dataframe and create new files
def break_and_create(location, n):
    data = pd.read_csv(location, header=None)
    #Number of observations in the data file
    shape = int(data.shape[1])
    #Number of divisions to be made
    div = 0
    if shape%1000 == 0:
        div = int(shape/1000)
    else:
        div = int(shape/1000)+1
    #Breaking each file and saving to a a new folder
    start = 0
    end = 1000
    count = n
    for i in range(div):
        if end <= shape:
            new_data = data.iloc[:,start:end]
            save_location = "E:/EEG Analysis/train_molecular/"+str(count)+".csv"
            new_data.to_csv(save_location)
            start = start+1000
            end = end+1000
            count = count+1
        else:
            new_data = data.iloc[:,start:]
            new_data = new_data.values
            new_data = pad_sequences(new_data, padding='post', maxlen=1000)
            save_location = "E:/EEG Analysis/train_molecular/"+str(count)+".csv"
            new = pd.DataFrame(new_data)
            new.to_csv(save_location)
            count = count+1
    return count

In [56]:
#create new labels and data set
count = 0
labels = []
pbar = tqdm_notebook(total=data.shape[0])
for i in range(data.shape[0]):
    old = count
    count = break_and_create(data['locations'][i] ,count)
    for j in range(old, count):
        labels.append(data['label'][i])
    pbar.update(1)
pbar.close

<bound method tqdm_notebook.close of 508/|/100%|| 508/508 [3:51:00<00:00, 32.27s/it]>